## Conditional GAN

In [1]:
import numpy as np
import matplotlib.pyplot as plt

### MNIST dataset

In [2]:
data = np.genfromtxt('train.csv', delimiter=',',skip_header=1)
# normalize X
X = (data[:,1:]/255).reshape(-1,28,28,1)
# one hot encode y
y = np.zeros(shape=(len(X),10),dtype=int)
y[np.arange(len(data)),data[:,0].astype(int)] = 1
# subsample X (for faster training)
X_copy = X.copy()
mask = np.random.choice(range(len(X)),size=2000)
X = X[mask]

In [3]:
from keras.models import Model, Sequential
from keras.layers import Convolution2D, LeakyReLU, BatchNormalization, Flatten, Concatenate
from keras.layers import Dense, Input, Reshape, ReLU, Conv2DTranspose, Activation, Reshape

Using TensorFlow backend.


### Generate models

In [ ]:
def build_models():
    '''Generate keras models for a functional DCGAN'''
    # discriminator
    input_layer = Input(shape = X[0].shape)
    x = Convolution2D(filters=32, kernel_size=(5, 5), strides=(2, 2), padding='same')(input_layer)
    x = LeakyReLU(0.2)(x)
    x = Convolution2D(filters=64, kernel_size=(5, 5), strides=(2, 2), padding='same')(x)
    x = BatchNormalization(momentum=0.5)(x)
    x = LeakyReLU(0.2)(x)
    x = Convolution2D(filters=128, kernel_size=(5, 5), strides=(2, 2), padding='same')(x)
    x = BatchNormalization(momentum=0.5)(x)
    x = LeakyReLU(0.2)(x)
    x = Flatten()(x)
    input_label = Input(shape=(1,1,10))
    il = Reshape((10,))(input_label)
    x = Concatenate()([x, il])
    output = (Dense(1,activation='sigmoid'))(x)

    discriminator = Model(inputs=[input_layer,input_label],outputs=output)
    discriminator.compile(optimizer='adam',loss='binary_crossentropy')

    # generator
    input_layer = Input(shape=(1,1,100))
    input_label = Input(shape=(1,1,10))
    x = Concatenate()([input_layer, input_label])
    x = Dense(2048)(x)
    x = Reshape(target_shape=(4, 4, 128))(x)
    x = BatchNormalization(momentum=0.5)(x)
    x = ReLU()(x)
    x = Conv2DTranspose(filters=64,kernel_size=(5, 5),strides=(2, 2),padding='same',output_padding=(0,0))(x)
    x = BatchNormalization(momentum=0.5)(x)
    x = ReLU()(x)
    x = Conv2DTranspose(filters=32, kernel_size=(5, 5), strides=(2, 2), padding='same')(x)
    x = BatchNormalization(momentum=0.5)(x)
    x = ReLU()(x)
    x = Conv2DTranspose(filters=1, kernel_size=(5, 5), strides=(2, 2), padding='same')(x)
    x = BatchNormalization(momentum=0.5)(x)
    output = Activation('tanh')(x)

    generator = Model(inputs = [input_layer,input_label],outputs = output)
    generator.compile(optimizer='adam',loss='binary_crossentropy')

    input_layer = Input((1,1,100))
    input_label = Input((1,1,10))
    image = generator([input_layer, input_label])
    discriminator.trainable = False
    verdict = discriminator([image, input_label])
    gan = Model(inputs=[input_layer, input_label], outputs=verdict)
    gan.compile(optimizer='adam',loss='binary_crossentropy')
    
    return discriminator, generator, gan

### Training models with labels

In [ ]:
# initialize models
discriminator, generator, gan = build_models()

# setup parameters
epochs = 200
print_fq = 20
batch_size = 100
generator_ratio = 2

for epoch in range(epochs):
    if ((epoch+1)%print_fq==0):
        print('Epoch: ',epoch+1)
    for batch in range(len(X)//batch_size):
        # normalize data to -1, 1 (works better that way, also in line with tanh output of generator)
        x_b = X[batch*batch_size:(batch+1)*batch_size]*2-1
        y_b = y[batch*batch_size:(batch+1)*batch_size]
        # randomly generate latent values
        z = np.random.normal(size=((len(x_b),)+(1,1,100)))
        images = generator.predict([z,y_b.reshape((-1,1,1,10))])
        # add noise to images
        x_b = x_b + np.random.normal(scale=0.2,size=x_b.shape)
        x_b = np.clip(x_b,0,1)
        images = images + np.random.normal(scale=0.2,size=images.shape)
        images = np.clip(images,0,1)
        # generate labels for discriminator
        real = np.ones(len(x_b)) - np.random.random(size=len(x_b))*0.2
        fake = np.zeros(len(x_b)) + np.random.random(size=len(x_b))*0.2
        
        # train discriminator once
        discriminator.trainable = True
        discriminator.train_on_batch([x_b,y_b.reshape((-1,1,1,10))],real)
        discriminator.train_on_batch([images,y_b.reshape((-1,1,1,10))],fake)
        
        # train generator n times
        for i in range(generator_ratio):
            discriminator.trainable = False
            # generate new images
            z = np.random.normal(size=((len(x_b),)+(1,1,100)))
            # discriminator should be fooled into accepting those images
            fool = np.ones(len(x_b)) - np.random.random(size=len(x_b))*0.2
            gan.train_on_batch([z,y_b.reshape((-1,1,1,10))],fool)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


### Generated images

In [ ]:
z = np.random.normal(size=(5,1,1,100))

im = generator.predict(z)

fig, ax = plt.subplots(ncols=5,figsize=(16,6))
for i in range(5):
    ax[i].imshow(im[i].reshape(28,28),cmap='gray')
plt.show()